In [17]:
%matplotlib inline

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FuncAnimation


class RollingShutterSensor:
    def __init__(self, sensor_height, fps, exposure_time, readout_time, led_pulses, led_rows):
        self.sensor_height = sensor_height
        self.fps = fps
        self.exposure_time = exposure_time
        self.readout_time = readout_time
        self.led_pulses = pd.read_csv(led_pulses)
        self.led_rows = led_rows

        self.update_read_times()

    def update_read_times(self):
        self.sensor_row_read_times = np.linspace(0, self.readout_time * self.sensor_height, self.sensor_height)
        self.exposure_starts = self.sensor_row_read_times
        self.exposure_ends = self.sensor_row_read_times + self.exposure_time

    def plot_sensor_readout(self, ax, duration=2,  time_offset=0):
        frame_time = 1 / self.fps
        num_frames = int(duration / frame_time)

        batch_size = 10  # Number of consecutive rows to plot in each batch
        skip_size = 20   # Number of rows to skip between batches

        for batch_start in range(0, self.sensor_height, batch_size + skip_size):
            # Determine the end of the current batch
            batch_end = min(batch_start + batch_size, self.sensor_height)

            # plot grey pulses  
            for index, led_pulse in self.led_pulses.iterrows():
                led_on_time = led_pulse['start_time']
                led_off_time = led_on_time + led_pulse['duration']
                for sensor_row in range(batch_start, batch_end):
                    if sensor_row in self.led_rows:
                        ax.plot([led_on_time, led_off_time], [sensor_row, sensor_row], 'grey', linewidth=1)
        
        # Plot each sensor_row's exposure window
        
            for frame in range(num_frames):
                frame_offset = (frame-1) * frame_time
                for sensor_row in range(batch_start, batch_end):
                    start_time = self.exposure_starts[sensor_row] + frame_offset + time_offset
                    end_time = self.exposure_ends[sensor_row] + frame_offset + time_offset
                    
                    # Plot LED illumination for affected rows
                    if sensor_row in self.led_rows:
                        for index, led_pulse in self.led_pulses.iterrows():
                            led_on_time = led_pulse['start_time']
                            led_off_time = led_on_time + led_pulse['duration']
                                            
                            # Find overlap between LED illumination and sensor_row exposure window
                            overlap_start = max(led_on_time, start_time)
                            overlap_end = min(led_off_time, end_time)
                            
                            # Plot segment if there's an overlap
                            if overlap_start < overlap_end:
                                #ax.plot([overlap_start, overlap_end], [sensor_row, sensor_row], 'r', linewidth=10)
                                ax.plot([start_time, end_time], [sensor_row, sensor_row], 'r', linewidth=1)
                            else:
                                #ax.plot([start_time, end_time], [sensor_row, sensor_row], 'b', linewidth=1)
                                pass
                    else:
                        ax.plot([start_time, end_time], [sensor_row, sensor_row], 'b', linewidth=1)

    def animate_sensor_readout(self, duration=2, start_offset=0.0, end_offset=0.04, step=0.005):
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.set_xlim([0, duration])
        ax.set_ylim([0, self.sensor_height])
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('sensor_row Position (Subset)')

        def animate(offset):
            ax.clear()
            self.plot_sensor_readout(ax=ax, duration=duration, time_offset=offset)
            ax.grid(True)
            ax.set_title(f'Rolling Shutter Sensor Exposure over {duration} second with LED Signal - Offset: {offset:.4f}')
            
        anim = FuncAnimation(fig, animate, frames=np.arange(start_offset, end_offset, step), interval=50, repeat=False)
        
        plt.close(fig)  # Prevents duplicate display
        return anim

# Example of using the class
led_pulses = '/Users/lucbusquin/Projects/RMS-Contrail/pulses.csv'
led_rows = range(20, 50)
sensor = RollingShutterSensor(sensor_height=100, fps=25, exposure_time=1/40, readout_time=.03/720,
                              led_pulses=led_pulses, led_rows=led_rows)
# sensor.plot_sensor_readout(time_offset=0.0015)
anim = sensor.animate_sensor_readout()
from IPython.display import HTML
# HTML(anim.to_html5_video())
HTML(anim.to_jshtml())
# anim.to_html5_video()